# Smoke Test
Quick sanity check that all algorithms run without errors.

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('.')), ''))
# When running from notebooks/ directory, add parent to path
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.insert(0, os.path.join(os.getcwd(), '..'))
import numpy as np
print(f'Python: {sys.version}')
print(f'NumPy: {np.__version__}')
print(f'CWD: {os.getcwd()}')

In [ ]:
from models import (
    BatchSP2, BatchSP2Erasure, BatchTPG, BatchTPGNew,
    BatchSP2Simplified, BatchSP2RRR, BatchSGreedy, BatchTPGOld,
    calculate_repetitions, init_bandit,
    FEEDBACK_NONE, FEEDBACK_BEACON, FEEDBACK_ACK_SUCCESS,
)
from runners import run_episodes_with_same_erasures
from utils import generate_erasure_sequence_multi
print('All imports OK')

## 1. Unit test: single algorithm instantiation + run

In [ ]:
# Minimal config
K, M, T = 5, 3, 500
eps = np.array([0.3, 0.7, 0.95])
alphas = calculate_repetitions(eps, T, m=M)
mu = [0.9, 0.5, 0.3, 0.1, 0.7]
seq = generate_erasure_sequence_multi(T, M, eps)
print(f'alphas = {alphas}')
print(f'erasure_seq shape = {seq.shape}')

In [ ]:
# Test SP2 (no feedback baseline)
sp2 = BatchSP2(K, M, T, alphas, mu=mu, epsilon=eps,
               erasure_seq=seq, feedback_mode=FEEDBACK_NONE)
sp2.run()
reg_sp2 = np.cumsum(np.sum(sp2.regrets, axis=1))[-1]
print(f'SP2 final regret: {reg_sp2:.1f}')

In [ ]:
# Test SP2-Feedback (ACK on success)
sp2fb = BatchSP2Erasure(K, M, T, alphas, mu=mu, epsilon=eps,
                        erasure_seq=seq, feedback_mode=FEEDBACK_ACK_SUCCESS)
sp2fb.run()
reg_sp2fb = np.cumsum(np.sum(sp2fb.regrets, axis=1))[-1]
print(f'SP2-Feedback final regret: {reg_sp2fb:.1f}')

In [ ]:
# Test TPG (ACK on success)
tpg = BatchTPG(K, M, T, alphas, mu=mu, epsilon=eps,
               erasure_seq=seq, feedback_mode=FEEDBACK_ACK_SUCCESS)
tpg.run()
reg_tpg = np.cumsum(np.sum(tpg.regrets, axis=1))[-1]
print(f'TPG final regret: {reg_tpg:.1f}')

## 2. Integration test: run_episodes_with_same_erasures

In [ ]:
algs = [
    ('SAE', True, 'Scheduled'),
    ('SAE', True, 'Feedback', FEEDBACK_ACK_SUCCESS),
    ('SAE', True, 'TPG', FEEDBACK_ACK_SUCCESS),
]

K, M, T, EPISODES = 5, 3, 300, 3
eps_vec = np.array([0.3, 0.7, 0.95])

results = run_episodes_with_same_erasures(
    algs,
    iters=T,
    k=K,
    episodes=EPISODES,
    m=M,
    var=1,
    mu='random',
    eps=eps_vec,
    base_actions=np.zeros(M, dtype=int),
    feedback_mode=[a[3] if len(a)==4 else FEEDBACK_NONE for a in algs],
    rng_seed=42,
)

print('\n=== Summary ===')
for alg, data in results.items():
    reg = np.cumsum(np.sum(data['regret'], axis=1))[-1]
    print(f"{data['name']:>15s}: R_T={reg:.1f}, TX={data['avg_tx'][-1]:.0f}, FB={data['avg_fb'][-1]:.0f}")
print('\nAll good!')

## 3. Scaling test: K=50, M=4

In [ ]:
K_big, M_big, T_big = 50, 4, 2000
eps_big = np.array([0.3, 0.6, 0.8, 0.95])
alphas_big = calculate_repetitions(eps_big, T_big, m=M_big)
print(f'K={K_big}, M={M_big}, T={T_big}, alphas={alphas_big}')

results_big = run_episodes_with_same_erasures(
    algs,
    iters=T_big,
    k=K_big,
    episodes=2,
    m=M_big,
    var=1,
    mu='random',
    eps=eps_big,
    base_actions=np.zeros(M_big, dtype=int),
    feedback_mode=[a[3] if len(a)==4 else FEEDBACK_NONE for a in algs],
    rng_seed=42,
)

print('\n=== K=50 Summary ===')
for alg, data in results_big.items():
    reg = np.cumsum(np.sum(data['regret'], axis=1))[-1]
    print(f"{data['name']:>15s}: R_T={reg:.1f}")
print('K=50 test passed!')